Useful functions for this program: PIL.Image, scipy.ndimage, scipy.misc

Useful links:

http://scikit-learn.org/stable/auto_examples/applications/face_recognition.html

https://github.com/rndbrtrnd/udacity-deep-learning/blob/master/1_notmnist.ipynb

http://www.scipy-lectures.org/advanced/image_processing/index.html#opening-and-writing-to-image-files


In [ ]:
import pandas, numpy, scipy.ndimage
import os
import matplotlib.pyplot as plt
%matplotlib inline

import random
import math
from IPython.display import display, Image
import PIL.Image
import scipy.misc

In [ ]:
path = 'C:/Users/OPEN/Documents/NanZHAO/Projet CIR Fraude/data/store/'  
home = 'C:/Users/OPEN/Documents/NanZHAO/Projet CIR Fraude/'  

In [ ]:
def disp_sample_Folder(folder_name, sample_size):
    images = os.listdir(folder_name)
    print ("There are %d images for all the collected annonces. " %len(images))
    images_path = [os.path.join(folder_name, image) for image in images]
    images_sample = random.sample(images_path, sample_size)
    for image_sample in images_sample:
        i = Image(filename = image_sample)
        display(i)

In [ ]:
def sample_images_fun_1(folder_name, sample_size):
    images = os.listdir(folder_name)
    print ("There are %d images for all the collected annonces. " %len(images))
    images_path = [os.path.join(folder_name, image) for image in images]
    images_sample = random.sample(images_path, sample_size)
    return (images_sample)

A demonstration for the function sample_images_fun_1, use scipy.ndimage to get a image's information.

In [ ]:
sampled = sample_images(path, 10)
for s in sampled:
    height, width, channels = scipy.ndimage.imread(s).shape
    print(height, width, channels)

The first basic function to get height, width, channels information for all the collected images.
In the real word, this function is executed by a multi-processing program. The result of the multi-processing program is saved as 'images_info.json' in the data folder.

In [ ]:
image_info = pandas.DataFrame(columns = ('Id','postId', 'height', 'width', 'channels'))
images_all = os.listdir(path)

for im in images_all:
    postId = im.split('_')[0]
    Id = im.split(postId+'_')[1]
    shapes = scipy.ndimage.imread(os.path.join(path, im)).shape
    height = shapes[0]
    width  = shapes[1]
    if len(shapes) == 3:
        channels = shapes[2]
    else:
        channels = 2
    image_info = image_info.append(pandas.Series({'Id':Id, 'postId':postId, 'height':height, 'width':width, 'channels':channels}), ignore_index=True)


In [ ]:
data = pandas.read_json(os.path.join(home, 'data/images_info.json'))
print(data.head())

There exits images that do not have the mode 'RGB'

In [ ]:
data_2d = data[data['channels'] < 3]
print(len(data_2d))

Show images in the mode BW. There are some images corresponding to the term telephone.

In [ ]:
for i in data_2d.iterrows():
    i = Image(filename = (os.path.join(path, (str(i[1]['postId'])+'_'+i[1]['Id']))))
    display(i)

This function is used to show the shape and size of a group of selected images.

In [ ]:
def show_images(df, sample_size):
    index = df.index.tolist()
    index_sample = random.sample(index, sample_size)
    for idx in index_sample:
        image_path = os.path.join(path, (str(df.loc[idx]['postId'])+'_'+df.loc[idx]['Id']))
        i = Image(filename = image_path)
        print(image_path)
        print(df.loc[idx]['height'], df.loc[idx]['width'])
        print(os.path.getsize(image_path))
        display(i)

The steps below are used to display the distribution of the images pixel(height and width separately). 

In [ ]:
def rice_interval(data, column):
    len_interval = round(math.pow(len(data), 1/3)*2)
    return (round((max(data[column])-min(data[column]))/len_interval))

In [ ]:
rice_interval(data, 'height')

In [ ]:
n, bins, patches = plt.hist(data['height'], bins=3, facecolor='green', alpha=0.75)

plt.xlabel('Images height')
plt.ylabel('Frequency')
plt.title('Image Height Distribution')
plt.grid(True)
plt.show()

In [ ]:
rice_interval(data, 'width')

In [ ]:
n, bins, patches = plt.hist(data['width'], bins=5, facecolor='green', alpha=0.75)

plt.xlabel('Images width')
plt.ylabel('Frequency')
plt.title('Image Width Distribution')
plt.grid(True)
plt.show()

From the steps above, the result show that the collected data do not have the same pixels (width x height). There exist some small images (for exmaple 50x50). We have to uniform the pixels of all the images in order to apply PCA and clustering treatments. At the first time, a small group with the height in the intervals (0, 150], (150, 300] and (300, 450] are sampled to verify whether the reshaping over images will bring a lot of dammages or not.

In [ ]:
data_150 = data[data['height'] <= 150]
data_300 = data[(data.height >150) & (data.height <= 300)]
data_450 = data[(data.height >300) & (data.height <= 450)]


def sample_images(ls, df, sample_size):
    sampled = random.sample(df.index.tolist(), sample_size)
    for i in sampled:
        ls.append(i)
    return(ls)

For each of the sub-group, at the first time, we just take 10 images.

In [ ]:
index_sample = []
index_sample = sample_images(index_sample, data_150, 10)
index_sample = sample_images(index_sample, data_300, 10)
index_sample = sample_images(index_sample, data_450, 10)
data_sample = data.loc[index_sample]
print(data_sample.head())

The below function realize tow steps: 

1. to reshape a image and save to the folder reduced

2. to change the mode RGB to BW with the function numpy.array object.convert('L')

When we treate the reshape step (a PIL.Image.Image object applies resize(pixel, PIL.Image.ANTIALIAS), we use the filter PIL.Image.ANTIALIAS. There are other filters as PIL.Image.BILINEAR,  PIL.Image.BICUBIC and NONE.

For a PIL.Image.Image object, to get pixel .size, to get mode, .mode

We can also use function scipy.misc.imresize(images_400, 0.88) to reduce the pixel of an image.

In [ ]:
def reduce_bw_images(data_sample, pixel):
    reduced_folder = os.path.join(home, 'data/reduced/')
    bw_folder = os.path.join(home, 'data/bw/')
    if not os.path.exists(reduced_folder):
        os.makedirs(reduced_folder)
    if not os.path.exists(bw_folder):
        os.makedirs(bw_folder)
    
    for i in data_sample.index:
        image_path_relative = str(data_sample.loc[i, 'postId']) +'_'+data_sample.loc[i, 'Id']
        image_path = os.path.join(path, image_path_relative)
        print(image_path)
        img = PIL.Image.open(image_path)
        img_reshaped = img.resize(pixel, PIL.Image.ANTIALIAS)
        scipy.misc.imsave(os.path.join(reduced_folder, image_path_relative), img_reshaped)
        img_reshaped_bw = numpy.asarray(img_reshaped.convert('L'))
        scipy.misc.imsave(os.path.join(bw_folder, image_path_relative), img_reshaped_bw)
        to_display = PIL.Image.fromarray(img_reshaped_bw)
        display(to_display)
reduce_bw_images(data_sample, (150,150))